In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
df = pd.read_csv('/Users/brandon/Desktop/MILO/test_files/dummy_train_column_scan_comp.csv')
df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN

In [42]:
int(True)
target='outcome'
list(df[target].astype('string').unique())


ValueError: StringArray requires a sequence of strings or pandas.NA

In [62]:


def row_processing(dataframe, row_option, target):
    df = pd.DataFrame(dataframe)
    if (row_option == 0):
        missing = df[df.isna().any(axis=1)]
        output = df.drop(missing)
    elif (row_option == 1):
        imp_mean = IterativeImputer(random_state=0)
        X = df.drop(columns=[target])
        y = df[target]
        imp_mean.fit(X)
        result = pd.DataFrame(imp_mean.transform(X),columns=X.columns)
        result[target] = y
    return result
    

row








result

,va_a,va_b,va_c,cat_b,"('cat_a_one',)","('cat_a_three',)","('cat_a_two',)",outcome
0,0.751,0.869,0.460230,0.0,1.0,0.0,0.0,1
1,0.686,0.589,0.453551,1.0,0.0,0.0,1.0,0
2,0.303,0.385,0.463743,0.0,0.0,1.0,0.0,1
3,0.740,0.383,0.455292,1.0,1.0,0.0,0.0,0
4,0.048,0.662,0.458445,0.0,0.0,0.0,1.0,1
5,0.572,0.390,0.458562,1.0,0.0,1.0,0.0,0
6,0.534,0.507,0.944000,0.0,1.0,0.0,0.0,1
7,0.626,0.428,0.171000,1.0,0.0,0.0,1.0,0
8,0.114,0.812,0.596000,0.0,0.0,1.0,0.0,1
9,0.849,0.326,0.044000,1.0,1.0,0.0,0.0,0


In [3]:




enc = OneHotEncoder(handle_unknown='ignore')

#Need 2D vector to do the fit
vector = invalid[col_data['name']].values.reshape(-1,1)
enc.fit(vector)

trans = enc.transform(vector).toarray()

temp_df = pd.DataFrame(trans, columns=enc.categories_).add_prefix(col_data['name'] + '_').astype('int')

#Adjust original columns
invalid = pd.concat([temp_df,invalid], axis=1)
invalid = invalid.drop(col_data['name'], axis=1)

NameError: name 'invalid' is not defined

In [43]:
target='va_b'
col_list = list(df.columns)
i = col_list.index(target)
reorder_list = col_list[:i] + col_list[i + 1:] + [target]
df = df[reorder_list]


['va_a', 'va_b', 'va_c', 'cat_a', 'cat_b', 'outcome']


,va_a,va_c,cat_a,cat_b,outcome,va_b
0,0.751,a,one,a,True,0.869
1,0.686,a,two,b,False,0.589
2,0.303,b,three,a,True,0.385
3,0.740,b,one,b,False,0.383
4,0.048,NaN,two,a,True,0.662
5,0.572,NaN,three,b,False,0.390
6,0.534,0.944,one,a,True,0.507
7,0.626,0.171,two,b,False,0.428
8,0.114,0.596,three,a,True,0.812
9,0.849,0.044,one,b,False,0.326


In [23]:
def one_hot_encode(series):
    working = pd.Series(series)
    working = working.dropna()
    index = working.index
    
    enc = OneHotEncoder(handle_unknown='ignore')
    vector = working.values.reshape(-1,1)
    enc.fit(vector)
    trans = enc.transform(vector).toarray()
    output = pd.DataFrame(trans, columns=enc.categories_, index=index).add_prefix(series.name + '_').astype('int')
    
    return output

one_hot_encode(df['va_c'])

,va_c_0.044,va_c_0.061,va_c_0.063,va_c_0.171,va_c_0.199,va_c_0.252,va_c_0.324,va_c_0.36,va_c_0.52,va_c_0.596,va_c_0.727,va_c_0.75,va_c_0.916,va_c_0.941,va_c_0.944,va_c_a,va_c_b
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
